<a href="https://colab.research.google.com/github/TALeonard/19ma573thomasleonard/blob/master/src/MA573_Project_Larger_Scale_IV_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are going to attempt to compute implied volatilities for the system. Import numpy, scipy.stats and pyplot for the sake of use, and clone in the proper files from my directory.

In [1]:
!git clone https://github.com/TALeonard/19ma573thomasleonard.git #Only run once!

Cloning into '19ma573thomasleonard'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 406 (delta 43), reused 0 (delta 0), pack-reused 286
Receiving objects: 100% (406/406), 1.50 MiB | 21.92 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [2]:
cd /content/19ma573thomasleonard/src/

/content/19ma573thomasleonard/src


In [0]:
import numpy as np
import scipy.optimize as so
import matplotlib.pyplot as plt
import csv
from european_options_class import VanillaOption
from gbm_1d_class_v02 import Gbm

Define the "default" parameters which are true for every asset. These include:

* Otype - this is 1 for all values (I am only looking at calls for parsimony)
* Maturity - dependant on which fileset I am using. For the 4/18/2019 set, this maturty is 21 days out (I acquired this data on 3-27-2019). This doesn't count the day itself, and I will only count business days, so this is  actually 16 days (or 16/252, to convert to years).
* Interest rate - again, dependent on fileset. For 4/18/2019, r = 2.45%.

For the other data, we have the remaining information - Spot price, strike price, and (for implied volatility computation) the Option Price.

In [0]:
r = 0.0245
otype = 1
maturity = 16/252

Import the data. Each dataset is separated by maturity date.

In [0]:
data_4_18 = np.loadtxt(open('calls_4_18_2019.csv',"rb"),
                       delimiter=",",skiprows=1,usecols=(2,3,4,5))

In this set, we have that

* `data_4_18[:,0]` is the spot price
* `data_4_18[:,1]` is the strike price
* `data_4_18[:,2]` is the option price (calculated from Bloomberg info)
* `data_4_18[:,3]` is the implied volatility (also sourced from Bloomberg).

In [0]:
num_row = data_4_18.shape[0]
gbm_list = []
option_list = []

for i in range(num_row):
  option1 = VanillaOption(otype = otype, strike = data_4_18[i,1],
                         maturity = maturity, market_price = data_4_18[:,2])
  
  option_list.append(option1)
  if data_4_18[i,0] != data_4_18[i-1,0]:
    gbm1 = Gbm(init_state = data_4_18[i,0],
            drift_ratio = r,
            vol_ratio = .1 #Initial Guess
            ) 
    gbm_list.append(gbm1)
  else:
    continue
  

In [0]:
#Expand a column for identifier of underlying stock.
np_data_4_18 = np.append(data_4_18, np.zeros([num_row,1]), axis = 1)

for i in range(num_row):
  np_data_4_18[i,4] = np.floor(i/5) + 1